In [10]:
import pandas as pd
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import col, pandas_udf, expr

In [2]:
spark = SparkSession.builder.appName("chaper-5").getOrCreate()

25/02/06 00:22:14 WARN Utils: Your hostname, codespaces-10f702 resolves to a loopback address: 127.0.0.1; using 10.0.0.198 instead (on interface eth0)
25/02/06 00:22:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/06 00:22:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Spark UDF

In [4]:
def cubed(s):
    return s ** 3

spark.udf.register("cubed", cubed, LongType())

spark.range(1, 9).createOrReplaceTempView("udf_test")

In [ ]:
spark.sql(
    """
        SELECT   id
               , cubed(id) AS id_cubed
          FROM   udf_test
    """
).show()

In [ ]:
spark.sql(
    """
        SELECT   s
          FROM   test1
         WHERE   s IS NOT NULL
           AND   strlen(s) > 1
    """
)

In [5]:
def cubed(a: pd.Series) -> pd.Series:
    return a * a * a

cubed_udf = pandas_udf(cubed, returnType=LongType())

In [ ]:
df = spark.range(1, 9)
df.select("id", cubed_udf(col("id")).alias("cubed_id")).show()

### External Data Sources

* PostgreSQL
* MySQL
* Snowflake

In [ ]:
# postgresql

# read1
jdbc_df1 = (
    spark
    .read
    .format("jdbc")
    .option("url", "jdbc:postgresql://[DBSERVER]")
    .option("dbtable", "[SCHEMA].[TABLENAME]")
    .option("user", "[USERNAME]")
    .option("password", "[PASSWORD]")
    .load()
)

# read2
jdbc_df2 = (
    spark
    .read
    .jdbc("jdbc:postgresql://[DBSERVER]", "[SCHEMA]:[TABLENAME]", properties={"user": "[USERNAME]", "password": ["PASSWORD"]})
)

# write1
(
    jdbc_df1
    .write
    .format("jdbc")
    .option("url", "jdbc:postgresql://[DBSERVER]")
    .option("dbtable", "[SCHEMA].[TABLENAME]")
    .option("user", "[USERNAME]")
    .option("password", "[PASSWORD]")
    .save()
)

# write2
(
    jdbc_df2
    .write
    .jdbc("jdbc:postgresql://[DBSERVER]", "[SCHEMA]:[TABLENAME]", properties={"user": "[USERNAME]", "password": ["PASSWORD"]})
)


In [ ]:
# mysql

# read
jdbc_df = (
    spark
    .read
    .format("jdbc")
    .option("url", "jdbc:mysql://[DBSERVER]:3306/[DATABASE]")
    .option("driver", "com.mysql.jdbcDriver")
    .option("dbtable", "[TABLENAME]")
    .option("user", "[USERNAME]")
    .option("password", "[PASSWORD]")
    .load()
)

# write
(
    jdbc_df
    .write
    .format("jdbc")
    .option("url", "jdbc:mysql://[DBSERVER]:3306/[DATABASE]")
    .option("driver", "com.mysql.jdbcDriver")
    .option("dbtable", "[TABLENAME]")
    .option("user", "[USERNAME]")
    .option("password", "[PASSWORD]")
    .save()
)

In [ ]:
# Reference
# https://docs.snowflake.com/ko/user-guide/spark-connector-use

sc = SparkContext("local", "chaper-5")
spark = SQLContext(sc)
spark_conf = SparkConf().setMaster("local").setAppName("chaper-5-snowflake-app")

sc._jsc.hadoopConfiguration().set("fs.s3n.awsAccessKeyId", "[AWS_KEY]")
sc._jsc.hadoopConfiguration().set("fs.s3n.awsSecretAccessKey", "[AWS_SECRET_KEY]")

# use password
sf_option = {
    "sfURL": "<account-identifier>.snowflakecomputing.com",
    "sfUser": "<user-name>",
    "sfPassword": "<password>",
    "sfDatabase": "<database>",
    "sfSchema": "<schema>",
    "sfWarehouse": "<warehouse>"
}

# use oauth token (recommand)
sf_option = {
    "sfURL": "<account-identifier>.snowflakecomputing.com",
    "sfUser": "<user-name>",
    "sfAuthenticator": "oauth",
    "sfToken": "<external-oauth-access-token>",
    "sfDatabase": "<database>",
    "sfSchema": "<schema>",
    "sfWarehouse": "<warehouse>"
}


SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

QUERY = ""

df = (
    spark
    .read
    .format(SNOWFLAKE_SOURCE_NAME)
    .option(**sf_option)
    .option("query", QUERY)
    .load()
)

df.show()

### Spark Build-In Functions

In [12]:
spark.sql(
    """
        SELECT array_distinct(array(1, 2, 3, null, 3))
    """
).show()

+---------------------------------------+
|array_distinct(array(1, 2, 3, NULL, 3))|
+---------------------------------------+
|                        [1, 2, 3, NULL]|
+---------------------------------------+



In [15]:
spark.sql(
    """
        SELECT array_intersect(array(1, 2, 3), array(1, 3, 5))
    """
).show()

+-----------------------------------------------+
|array_intersect(array(1, 2, 3), array(1, 3, 5))|
+-----------------------------------------------+
|                                         [1, 3]|
+-----------------------------------------------+



In [16]:
spark.sql(
    """
        SELECT array_union(array(1, 2, 3), array(1, 3, 5))
    """
).show()

+-------------------------------------------+
|array_union(array(1, 2, 3), array(1, 3, 5))|
+-------------------------------------------+
|                               [1, 2, 3, 5]|
+-------------------------------------------+



In [17]:
spark.sql(
    """
        SELECT array_except(array(1, 2, 3), array(1, 3, 5))
    """
).show()

+--------------------------------------------+
|array_except(array(1, 2, 3), array(1, 3, 5))|
+--------------------------------------------+
|                                         [2]|
+--------------------------------------------+



In [18]:
spark.sql(
    """
        SELECT array_join(array('hello', 'world'), ' ')
    """
).show()

+----------------------------------+
|array_join(array(hello, world),  )|
+----------------------------------+
|                       hello world|
+----------------------------------+



In [19]:
spark.sql(
    """
        SELECT array_max(array(1, 2, 3, null, 3))
    """
).show()

+----------------------------------+
|array_max(array(1, 2, 3, NULL, 3))|
+----------------------------------+
|                                 3|
+----------------------------------+



In [20]:
spark.sql(
    """
        SELECT array_min(array(1, 2, 3, null, 3))
    """
).show()

+----------------------------------+
|array_min(array(1, 2, 3, NULL, 3))|
+----------------------------------+
|                                 1|
+----------------------------------+



In [21]:
spark.sql(
    """
        SELECT array_position(array(3, 2, 1), 1)
    """
).show()

+---------------------------------+
|array_position(array(3, 2, 1), 1)|
+---------------------------------+
|                                3|
+---------------------------------+



In [23]:
spark.sql(
    """
        SELECT array_remove(array(1, 2, 3, null, 3), 3)
    """
).show()

+----------------------------------------+
|array_remove(array(1, 2, 3, NULL, 3), 3)|
+----------------------------------------+
|                            [1, 2, NULL]|
+----------------------------------------+



In [25]:
spark.sql(
    """
        SELECT arrays_overlap(array(1, 2, 3), array(3, 4, 5))
    """
).show()

+----------------------------------------------+
|arrays_overlap(array(1, 2, 3), array(3, 4, 5))|
+----------------------------------------------+
|                                          true|
+----------------------------------------------+



In [26]:
spark.sql(
    """
        SELECT array_sort(array('b', 'd', null, 'c', 'a'))
    """
).show()

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|array_sort(array(b, d, NULL, c, a), lambdafunction((IF(((namedlambdavariable() IS NULL) AND (namedlambdavariable() IS NULL)), 0, (IF((namedlambdavariable() IS NULL), 1, (IF((namedlambdavariable() IS NULL), -1, (IF((namedlambdavariable() < namedlambdavariable()), -1, (IF((namedlambdavariable() > namedlambdavariable()), 1, 0)))))))))), namedlambdavariable(), namedlambdavariable()))|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [27]:
spark.sql(
    """
        SELECT concat(array(1, 2, 3), array(3, 4, 5))
    """
).show()

+--------------------------------------+
|concat(array(1, 2, 3), array(3, 4, 5))|
+--------------------------------------+
|                    [1, 2, 3, 3, 4, 5]|
+--------------------------------------+



In [28]:
spark.sql(
    """
        SELECT flatten(array(array(1, 2, 3), array(3, 4, 5)))
    """
).show()

+----------------------------------------------+
|flatten(array(array(1, 2, 3), array(3, 4, 5)))|
+----------------------------------------------+
|                            [1, 2, 3, 3, 4, 5]|
+----------------------------------------------+



In [30]:
spark.sql(
    """
        SELECT array_repeat('123', 3)
    """
).show()

+--------------------+
|array_repeat(123, 3)|
+--------------------+
|     [123, 123, 123]|
+--------------------+



In [31]:
spark.sql(
    """
        SELECT reverse(array(2, 1, 4, 3))
    """
).show()

+--------------------------+
|reverse(array(2, 1, 4, 3))|
+--------------------------+
|              [3, 4, 1, 2]|
+--------------------------+



In [ ]:
spark.sql(
    """
        SELECT sequence(1, 5);
    """
).show()


+---------------+
| sequence(1, 5)|
+---------------+
|[1, 2, 3, 4, 5]|
+---------------+



In [36]:
spark.sql(
    """
        SELECT sequence(5, 1);
    """
).show()

+---------------+
| sequence(5, 1)|
+---------------+
|[5, 4, 3, 2, 1]|
+---------------+



In [37]:
spark.sql(
    """
        SELECT sequence(to_date('2018-01-01'), to_date('2018-03-01'), interval 1 month);
    """
).show()

+----------------------------------------------------------------------+
|sequence(to_date(2018-01-01), to_date(2018-03-01), INTERVAL '1' MONTH)|
+----------------------------------------------------------------------+
|                                                  [2018-01-01, 2018...|
+----------------------------------------------------------------------+



In [40]:
spark.sql(
    """
        SELECT shuffle(array(1, 2, null, 3))
    """
).show()

+-----------------------------+
|shuffle(array(1, 2, NULL, 3))|
+-----------------------------+
|              [2, 1, 3, NULL]|
+-----------------------------+



In [41]:
spark.sql(
    """
        SELECT slice(array(1, 2, 3, 4), -2, 2)
    """
).show()

+-------------------------------+
|slice(array(1, 2, 3, 4), -2, 2)|
+-------------------------------+
|                         [3, 4]|
+-------------------------------+



In [43]:
spark.sql(
    """
        SELECT arrays_zip(array(1, 2, 3, 4), array(5, 6, 7, 8), array(9, 10, 11, 12))
    """
).show()

+----------------------------------------------------------------------+
|arrays_zip(array(1, 2, 3, 4), array(5, 6, 7, 8), array(9, 10, 11, 12))|
+----------------------------------------------------------------------+
|                                                  [{1, 5, 9}, {2, 6...|
+----------------------------------------------------------------------+



In [46]:
spark.sql(
    """
        SELECT element_at(array(3, 1, 2), 3)
    """
).show()

+-----------------------------+
|element_at(array(3, 1, 2), 3)|
+-----------------------------+
|                            2|
+-----------------------------+



In [47]:
spark.sql(
    """
        SELECT cardinality(array('b', 'd', 'c', 'a'))
    """
).show()

+------------------------------+
|cardinality(array(b, d, c, a))|
+------------------------------+
|                             4|
+------------------------------+



In [49]:
spark.sql(
    """
        SELECT map_from_arrays(array(1.0, 3.0), array('2', '4'))
    """
).show()

+---------------------------------------------+
|map_from_arrays(array(1.0, 3.0), array(2, 4))|
+---------------------------------------------+
|                         {1.0 -> 2, 3.0 -> 4}|
+---------------------------------------------+



In [ ]:
spark.sql(
    """
        SELECT map_from_entries(array(struct(1, 'a'), struct(2, 'b')))
    """
).show()

+---------------------------------------------------+
|map_from_entries(array(struct(1, a), struct(2, b)))|
+---------------------------------------------------+
|                                   {1 -> a, 2 -> b}|
+---------------------------------------------------+



In [53]:
spark.sql(
    """
        SELECT map_concat(map(1, 'a', 2, 'b'), map(3, 'c', 4, 'd'))
    """
).show()

+--------------------------------------------+
|map_concat(map(1, a, 2, b), map(3, c, 4, d))|
+--------------------------------------------+
|                        {1 -> a, 2 -> b, ...|
+--------------------------------------------+



In [54]:
spark.sql(
    """
        SELECT element_at(map(1, 'a', 2, 'b'), 2)
    """
).show()

+------------------------------+
|element_at(map(1, a, 2, b), 2)|
+------------------------------+
|                             b|
+------------------------------+



In [55]:
spark.sql(
    """
        SELECT cardinality(map(1, 'a', 2, 'b'))
    """
).show()

+----------------------------+
|cardinality(map(1, a, 2, b))|
+----------------------------+
|                           2|
+----------------------------+



### Spark High-Order Functions

In [40]:
schema = StructType([StructField("celsius", ArrayType(IntegerType()))])

t_list = [[35, 36, 32, 30, 40, 42, 38]], [[31, 32, 34, 55, 56]]
t_c = spark.createDataFrame(t_list, schema)
t_c.createOrReplaceTempView("tC")

t_c.show()

+--------------------+
|             celsius|
+--------------------+
|[35, 36, 32, 30, ...|
|[31, 32, 34, 55, 56]|
+--------------------+



In [41]:
spark.sql(
    """
        SELECT   celsius
               , transform(celsius, t -> (((t * 9) div 5) + 32)) AS fahrenhelt
          FROM   tC
    """
).show()

+--------------------+--------------------+
|             celsius|          fahrenhelt|
+--------------------+--------------------+
|[35, 36, 32, 30, ...|[95, 96, 89, 86, ...|
|[31, 32, 34, 55, 56]|[87, 89, 93, 131,...|
+--------------------+--------------------+



In [42]:
spark.sql(
    """
        SELECT   celsius
               , filter(celsius, t -> t> 38) AS high
          FROM   tC
    """
).show()

+--------------------+--------+
|             celsius|    high|
+--------------------+--------+
|[35, 36, 32, 30, ...|[40, 42]|
|[31, 32, 34, 55, 56]|[55, 56]|
+--------------------+--------+



In [43]:
spark.sql(
    """
        SELECT   celsius
               , exists(celsius, t -> t = 38) AS threshold
          FROM   tC
    """
).show()

+--------------------+---------+
|             celsius|threshold|
+--------------------+---------+
|[35, 36, 32, 30, ...|     true|
|[31, 32, 34, 55, 56]|    false|
+--------------------+---------+



In [ ]:
spark.sql(
    """
        SELECT   celsius
               , reduce(
                      celsius
                    , 0
                    , (t, acc) -> t + acc
                    , acc -> (acc div size(celsius) * 9 div 5) + 32 
                 ) as avgFahrenheit
          FROM   tC
    """
).show()

+--------------------+-------------+
|             celsius|avgFahrenheit|
+--------------------+-------------+
|[35, 36, 32, 30, ...|           96|
|[31, 32, 34, 55, 56]|          105|
+--------------------+-------------+



25/02/06 01:52:49 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /tmp/blockmgr-12c46f37-bae7-4bec-9290-c474ab9b612b. Falling back to Java IO way
java.io.IOException: Failed to delete: /tmp/blockmgr-12c46f37-bae7-4bec-9290-c474ab9b612b
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:166)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:109)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:90)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively(SparkFileUtils.scala:121)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively$(SparkFileUtils.scala:120)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1126)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1(DiskBlockManager.scala:368)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1$adapted(DiskBlockManager.scala:364)
	at scala.collection.IndexedSeqOptimize

### Data Processing in Spark

In [14]:
trip_delays_file_path = "./datasets/flights/departuredelays.csv"
airports_na_file_path = "./datasets/flights/airport-codes-na.txt"

airports_na = (
    spark
    .read
    .format("csv")
    .options(header="true", inferSchema="true", sep="\t")
    .load(airports_na_file_path)
)
airports_na.createOrReplaceTempView("airports_na")

departure_delays = (
    spark
    .read.format("csv")
    .options(header="true")
    .load(trip_delays_file_path)
)
departure_delays = (
    departure_delays
    .withColumn("delay", expr("CAST(delay as INT) as delay"))
    .withColumn("distance", expr("CAST(distance as INT) as distance"))
)
departure_delays.createOrReplaceTempView("departure_delays")

foo = (
    departure_delays
    .filter(expr(
            """
                    origin == 'SEA'
                AND destination == 'SFO'
                AND date like '01010%'
                AND delay > 0
            """
        )
    )
)
foo.createOrReplaceTempView("foo")

In [12]:
spark.sql(
    """
        SELECT   *
          FROM   airports_na
         LIMIT   10
    """
).show()

+-----------+-----+-------+----+
|       City|State|Country|IATA|
+-----------+-----+-------+----+
| Abbotsford|   BC| Canada| YXX|
|   Aberdeen|   SD|    USA| ABR|
|    Abilene|   TX|    USA| ABI|
|      Akron|   OH|    USA| CAK|
|    Alamosa|   CO|    USA| ALS|
|     Albany|   GA|    USA| ABY|
|     Albany|   NY|    USA| ALB|
|Albuquerque|   NM|    USA| ABQ|
| Alexandria|   LA|    USA| AEX|
|  Allentown|   PA|    USA| ABE|
+-----------+-----+-------+----+



In [15]:
spark.sql(
    """
        SELECT   *
          FROM   departure_delays
         LIMIT   10
    """
).show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
|01030605|    0|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        ATL|
|01050605|    9|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+



In [16]:
spark.sql("SELECT * FROM foo").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



In [18]:
bar = departure_delays.union(foo)
bar.createOrReplaceTempView("foo")

bar.filter(expr(
    """
        origin == 'SEA'
        AND destination == 'SFO'
        AND date LIKE '01010%'
        AND delay > 0                
    """
)).show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



In [19]:
foo.join(
    airports_na,
    airports_na.IATA == foo.origin
).select("City", "State", "date", "delay", "distance", "destination").show()

+-------+-----+--------+-----+--------+-----------+
|   City|State|    date|delay|distance|destination|
+-------+-----+--------+-----+--------+-----------+
|Seattle|   WA|01010710|   31|     590|        SFO|
|Seattle|   WA|01010955|  104|     590|        SFO|
|Seattle|   WA|01010730|    5|     590|        SFO|
+-------+-----+--------+-----+--------+-----------+



In [ ]:
spark.sql("DROP TABLE IF EXISTS departure_delays_window")
spark.sql(
    """
        CREATE TABLE departure_delays_window AS (
            SELECT   origin
                   , destination
                   , SUM(delay) AS total_delays
              FROM   departure_delays
             WHERE   1=1
               AND   origin      IN ('SEA', 'SFO', 'JFK')
               AND   destination IN ('SEA', 'SFO', 'JFK', 'DEN', 'ORD', 'LAX', 'ATL')
             GROUP   
                BY   1, 2
        )      
    """
)
spark.sql("SELECT * FROM departure_delays_window")

In [ ]:
spark.sql(
    """
        SELECT   origin
               , destination
               , SUM(TotalDelays) as total_delays
          FROM   departure_delays_window
         WHERE   origin = '[ORIGIN]'
         GROUP
            BY   origin, destination
         ORDER
            BY   SUM(TotalDelays) DESC
         LIMIT   3
    """
)

In [ ]:
spark.sql(
    """
        WITH cte AS (
            SELECT   origin
                   , destination
                   , TotalDelays
                   , dense_rank() OVER (PARTITION BY origin ORDER BY TotalDelays DESC) AS rank
              FROM   departure_delays_window
        )
        SELECT   origin
               , destination
               , TotalDelays
               , rank
          FROM   cte
         WHERE   rank <= 3
    """
).show()

In [25]:
foo.show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



In [26]:
foo2 = (
    foo.withColumn(
        "status",
        expr("CASE WHEN delay <= 10 THEN 'On-Time' ELSE 'Delayed' END")
    )
)
foo2.show()

+--------+-----+--------+------+-----------+-------+
|    date|delay|distance|origin|destination| status|
+--------+-----+--------+------+-----------+-------+
|01010710|   31|     590|   SEA|        SFO|Delayed|
|01010955|  104|     590|   SEA|        SFO|Delayed|
|01010730|    5|     590|   SEA|        SFO|On-Time|
+--------+-----+--------+------+-----------+-------+



In [27]:
foo3 = foo2.drop("delay")
foo3.show()

+--------+--------+------+-----------+-------+
|    date|distance|origin|destination| status|
+--------+--------+------+-----------+-------+
|01010710|     590|   SEA|        SFO|Delayed|
|01010955|     590|   SEA|        SFO|Delayed|
|01010730|     590|   SEA|        SFO|On-Time|
+--------+--------+------+-----------+-------+



In [28]:
foo4 = foo3.withColumnRenamed("status", "flight_status")
foo4.show()

+--------+--------+------+-----------+-------------+
|    date|distance|origin|destination|flight_status|
+--------+--------+------+-----------+-------------+
|01010710|     590|   SEA|        SFO|      Delayed|
|01010955|     590|   SEA|        SFO|      Delayed|
|01010730|     590|   SEA|        SFO|      On-Time|
+--------+--------+------+-----------+-------------+



In [39]:
spark.sql("""
    WITH cte AS (
        SELECT   destination
               , CAST(SUBSTRING(date, 0, 2) AS INT) AS month
               , delay
          FROM   departure_delays
         WHERE   origin = 'SEA'
    )
    SELECT   *
      FROM   cte
     PIVOT   (
                CAST(AVG(delay) AS DECIMAL(4, 2)) AS AvgDelay,
                MAX(delay) AS MaxDelay 
                FOR month IN (1 JAN, 2 FEB)
             )
     ORDER   BY destination
""").show()

+-----------+------------+------------+------------+------------+
|destination|JAN_AvgDelay|JAN_MaxDelay|FEB_AvgDelay|FEB_MaxDelay|
+-----------+------------+------------+------------+------------+
|        ABQ|       19.86|         316|       11.42|          69|
|        ANC|        4.44|         149|        7.90|         141|
|        ATL|       11.98|         397|        7.73|         145|
|        AUS|        3.48|          50|       -0.21|          18|
|        BOS|        7.84|         110|       14.58|         152|
|        BUR|       -2.03|          56|       -1.89|          78|
|        CLE|       16.00|          27|        NULL|        NULL|
|        CLT|        2.53|          41|       12.96|         228|
|        COS|        5.32|          82|       12.18|         203|
|        CVG|       -0.50|           4|        NULL|        NULL|
|        DCA|       -1.15|          50|        0.07|          34|
|        DEN|       13.13|         425|       12.95|         625|
|        D